In [2]:
import pandas as pd
from preprocess import preprocess
from BTM_excute import BTM_excute
import tmplot as tmp

### Sender&Time&Non-topic Features

In [2]:
train = pd.read_csv('data/weibo_train_data.txt',sep='\t',
                    names=['uid','mid','time','forward_count','comment_count','like_count','content'],parse_dates=[2])
                    
def preprocess_excute(data,training_month,start,end):
    processor = preprocess(data,training_month,start,end)
    # SENDER FEATURES
    data = processor.post_frequency()
    data = train_sample = processor.fans_features()
    print('Send Features done!')
    # TIME FEATURES
    data = processor.time_features()
    print('Time Features done!')
    # EXPLICIT CONTENT FEATURES
    data = processor.explicit_content_features()
    print('Explicit Content Features done!')
    # IMPLICIT CONTENT FEATURES
    processor.tokenize_content() # tokenization AND remove stopwrods
    print('Tokenization done!')
    processor.TFIDF() # extract keywords by TF-IDF in DESCENDING order
    print('Keywords Extraction done!')
    processor.sentiment_analysis() # sentiment analysis: [-,+]
    print('Sentiment Analysis done!')
    return data

prepared_df = preprocess_excute(data=train,training_month=6,start='2015-02-01',end='2015-07-31')
prepared_df.drop(['seg_content_lis','unique_words'],axis=1,inplace=True)
#prepared_df.to_csv('data/cleaned_train.txt',sep=',',index=False)

KeyboardInterrupt: 

### Topic Modeling

In [3]:
prepared_df = pd.read_csv('data/cleaned_train.txt',parse_dates=[2])
prepared_df.seg_content = prepared_df.seg_content.fillna('空内容') # 填充去除停用词后、无实质内容的博文为“空内容”

#### preprocess

In [4]:
corpus = prepared_df['seg_content'].str.strip().tolist() # list[str]
BTM = BTM_excute(data=corpus)
BTM.preprocess()

Preprocess Results:

Vocabulary list: ['一一分' '一一列举' '一一对应' ... '龟裂' '龟速' '龟鳖'] 
 Total number of unique words: 348475
******************************
Word pairs and their frequencies:
   (0, 17470)	1
  (0, 172794)	1
  (0, 269791)	1
  (0, 218133)	1
  (0, 297777)	1
  (0, 229980)	1
  (0, 155755)	1
  (0, 163158)	1
  (0, 63015)	1
  (0, 238131)	1
  (0, 117684)	1
  (0, 9719)	1
  (0, 185183)	1
  (1, 146979)	1
  (1, 130375)	1
  (1, 146819)	1
  (1, 235891)	1
  (2, 269791)	2
  (2, 218133)	1
  (2, 297777)	1
  (2, 229980)	1
  (2, 155755)	1
  (2, 238131)	1
  (2, 210487)	1
  (2, 248120)	1
  :	:
  (1225083, 115552)	1
  (1225083, 7678)	1
  (1225083, 93043)	1
  (1225083, 133886)	1
  (1225084, 91675)	1
  (1225084, 54384)	1
  (1225085, 76255)	1
  (1225085, 222563)	1
  (1225085, 242507)	1
  (1225085, 98594)	1
  (1225085, 148583)	1
  (1225085, 1024)	1
  (1225086, 257795)	4
  (1225086, 255067)	1
  (1225086, 157121)	1
  (1225086, 133282)	1
  (1225086, 264145)	1
  (1225086, 107163)	2
  (1225086, 229280)	1
  (12

#### training

##### T=10

In [29]:
model = BTM.modeling(topic_num=10,wordz_size=20,alpha=50/10,beta=0.01,iterations=50,seed=12321)

100%|██████████| 50/50 [09:10<00:00, 11.01s/it]

Training done~


In [30]:
doc_topic_df = BTM.inference()
doc_topic_df

100%|██████████| 1225088/1225088 [00:21<00:00, 56061.11it/s]


Topic inference done~


,seg_content,topic
0,丽江 旅游 股票 炒股 财经 理财 投资 推荐 包赢 股 盈利 对半分 成 不算 本金 群,8
1,想开 幸福 想不开 痛苦,1
2,浩丰 科技 股票 股神 股市 炒股 财经 理财 投资 股票 庄家 最低 资金 万元 做庄 每...,8
3,女性 占便宜 从来不 生气 心态,1
4,网宿 科技 股票 炒股 财经 理财 投资 推荐 包赢 股 盈利 对半分 成 不算 本金 群,8
...,...,...
1225083,开通 地址 下载 客户端,7
1225084,分享 图片,2
1225085,可爱 太 懂事 一天到晚 睡觉 爸爸 强 多少倍,4
1225086,陈明 永 红包 羊年 未 好运 先来 抢个 红包 乐翻天 陈明 永 粉丝 红包 里 抽到 现...,9


In [31]:
doc_topic_df.to_csv('data/topics10/doc_topics_10.txt',sep=',',encoding='gbk',index_label='doc_id')

In [32]:
# Visualization
tmp.report(model=model, docs=corpus)

c:\Users\MarkWu\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\manifold\_t_sne.py:780: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
c:\Users\MarkWu\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\manifold\_t_sne.py:790: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(
c:\Users\MarkWu\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\manifold\_t_sne.py:819: FutureWarning: 'square_distances' has been introduced in 0.24 to help phase out legacy squaring behavior. The 'legacy' setting will be removed in 1.1 (renaming of 0.26), and the default setting will be changed to True. In 1.3, 'square_distances' will be removed altogether, and distances will be squared by default. Set 'square_distances'=True to silence this warning.
  warnings.warn(


In [ ]:
# Topic 0:影视追星
# Topic 1:未知
# Topic 2:商业广告
# Topic 3:学习工作
# Topic 4:烹饪料理
# Topic 5:商业广告（？）
# Topic 6:党建/政策
# Topic 7:?
# Topic 8:投资理财
# Topic 9:?
doc_topic_df.topic.astype(int)
doc_topic_df.loc[doc_topic_df.topic==3]

,seg_content,topic
78,申请 微博 认证 认证 信息 远洋 股票投资 公司 经理 请,3
101,发表 博文 数字 域名 结拍元 数字 域名 结拍元 精品 数字 便宜 备案 域名 备案 域名...,3
148,发表 博文 综合 双拼 域名 建 信息 分类 网站 月 日讯据 爆料 月初 曝 交易 双拼 ...,3
281,秒 懂 区别,3
376,大脑 高效 工作 黄金 时段,3
...,...,...
1225057,平台 丢 台 机器 业务 运行 感些 华为 弟兄 修好 虚拟 华 平台 丢 数据 上次 强多...,3
1225068,空内容,3
1225070,想 本机 装个 集群 求问 经验之谈 海峰,3
1225078,打造 中国 传统 文化 优秀 平台 共建 全球华人 精神家园 才府 家园 欢迎您 优秀 传统...,3


##### T=20

In [5]:
model = BTM.modeling(topic_num=20,wordz_size=20,alpha=50/20,beta=0.01,iterations=50,seed=12321)

100%|██████████| 50/50 [17:52<00:00, 21.44s/it]


Training done~


In [6]:
doc_topic_df = BTM.inference()
doc_topic_df

100%|██████████| 1225088/1225088 [00:27<00:00, 44108.27it/s]


Topic inference done~


,seg_content,topic
0,丽江 旅游 股票 炒股 财经 理财 投资 推荐 包赢 股 盈利 对半分 成 不算 本金 群,16
1,想开 幸福 想不开 痛苦,11
2,浩丰 科技 股票 股神 股市 炒股 财经 理财 投资 股票 庄家 最低 资金 万元 做庄 每...,16
3,女性 占便宜 从来不 生气 心态,11
4,网宿 科技 股票 炒股 财经 理财 投资 推荐 包赢 股 盈利 对半分 成 不算 本金 群,16
...,...,...
1225083,开通 地址 下载 客户端,0
1225084,分享 图片,3
1225085,可爱 太 懂事 一天到晚 睡觉 爸爸 强 多少倍,8
1225086,陈明 永 红包 羊年 未 好运 先来 抢个 红包 乐翻天 陈明 永 粉丝 红包 里 抽到 现...,19


In [8]:
doc_topic_df.to_csv('data/topics10/doc_topics_20.txt',sep=',',encoding='gbk',index_label='doc_id')
#prepared_df = prepared_df.merge(doc_topic_df) # combine the topic inference and raw data
#prepared_df

In [7]:
# Visualization
tmp.report(model=model, docs=corpus)
# 0=学习活动；1=人文历史；2=流行影视&流行歌曲；3=婚庆广告&公司培训；4=婚庆广告？；5=追星；6=时政；7=编程/数学/技术交流；8=情绪疏泄(?)；9=烹饪料理；10=

c:\Users\MarkWu\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\manifold\_t_sne.py:780: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
c:\Users\MarkWu\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\manifold\_t_sne.py:790: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(
c:\Users\MarkWu\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\manifold\_t_sne.py:819: FutureWarning: 'square_distances' has been introduced in 0.24 to help phase out legacy squaring behavior. The 'legacy' setting will be removed in 1.1 (renaming of 0.26), and the default setting will be changed to True. In 1.3, 'square_distances' will be removed altogether, and distances will be squared by default. Set 'square_distances'=True to silence this warning.
  warnings.warn(


### Data Merge

In [9]:
import pandas as pd
import numpy as np
raw_df = pd.read_csv('data/cleaned_train.txt',sep=',',parse_dates=[2])
doc_topic_df = pd.read_csv('data/topics20/doc_topics_20.txt',sep=',',encoding='gbk')
#np.savetxt('data/user_features/mid_train.txt',raw_df.uid.unique(),fmt='%s') # 提取独立uid
#np.savetxt('data/user_features/mid_train.txt',raw_df.mid.unique(),fmt='%s') # 提取独立mid
mids = pd.read_csv('data/user_features/mid_train.txt',names=['mid'])
mids['doc_id'] = range(len(mids))
uids = pd.read_csv('data/user_features/uid_train.txt',names=['uid'])
uids['user_id'] = range(len(uids))
raw_df.drop(columns=['freq','seg_content'],inplace=True)
raw_df = raw_df.merge(mids)
raw_df = raw_df.merge(uids)


KeyError: "['seg_content'] not found in axis"

In [ ]:
raw_df.drop(columns=['uid','mid'],inplace=True)
raw_df = raw_df.merge(doc_topic_df)
raw_df
raw_df.to_csv('data/cleaned_train_with_topics20.txt',index=False)